In [61]:
from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit
provider = IBMProvider()
count_exp=job_exp.result().get_counts()
count_sim=job_sim.result().get_counts()
count_qiskit=job_qiskit.result().get_counts()

In [51]:
m = 4      # 0 -> 4 represent T=0.2 -> 1.0
N = 6

In [62]:
import numpy as np
from numpy import kron, matmul
from scipy.linalg import norm, expm

def list_kron(l) :
    ret = np.array([[1]])
    for i in range(len(l)) :
        ret = kron(ret, l[i])
    return ret

def calc_prob(is_chain, N, T) :
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    I = np.array([[1, 0], [0, 1]])
    zero = np.array([[1], [0]])
    H_ising = np.zeros((1<<N, 1<<N))
    for i in range(N - 1 if is_chain else N) :
        l = [I for j in range(N)]
        l[i] = Z
        l[(i + 1) % N] = Z
        H_ising = H_ising + list_kron(l)
    for i in range(N) :
        l = [I for j in range(N)]
        l[i] = X
        H_ising = H_ising + list_kron(l)
    
    l = [zero for j in range(N)]
    init = list_kron(l)
    
    final = matmul(expm(-1j * T * H_ising), init)
    prob = [np.abs(final[i][0]) ** 2 for i in range(1<<N)]
    return prob

def calc_dist(prob1, prob2):
    sum_error = 0
    for i in range(len(prob1)) :
        sum_error += abs(prob1[i] - prob2[i])
    return sum_error / 2



In [65]:
for m in range(5):
    # ideal_prob = calc_prob(True, N, 0.2 * (m + 1))
    ideal_prob = []
    counti=count_sim[m]
    for i in range(1<<N) :
        binary = bin(i)[2:].zfill(N)
        if binary in counti.keys() :
            ideal_prob.append(counti[binary]/4000)
        else :
            ideal_prob.append(0.)

    exp_prob = []
    counti=count_exp[m]
    for i in range(1<<N) :
        binary = bin(i)[2:].zfill(N)
        if binary in counti.keys() :
            exp_prob.append(counti[binary]/4000)
        else :
            exp_prob.append(0.)
    print(0.2 * (m + 1),calc_dist(ideal_prob, exp_prob))

0.2 0.24374999999999974
0.4 0.19424999999999965
0.6000000000000001 0.16124999999999995
0.8 0.17300000000000004
1.0 0.25049999999999983
